In [1]:
import pandas as pd
from io import StringIO
import requests

In [2]:
candidats = (
    pd
    .read_csv(
        'https://static.data.gouv.fr/resources/elections-legislatives-des-30-juin-et-7-juillet-2024-candidatures/20240619-161410/legislatives-2024-candidatures-france-entiere-tour-1-2024-06-19-17h10.csv',
    )
)

candidats

,Code département,Département,Code circonscription,Libellé circonscription,Numéro de panneau,N° dépôt,Sexe du candidat,Nom du candidat,Prénom du candidat,Date de naissance du candidat,Code nuance,Profession,Sortant,Sexe remplaçant,Nom remplaçant,Prénom remplaçant,Date de naissance remplaçant,Sortant remplaçant
0,1,Ain,101,1ère circonscription,1,2,M,LAHY,Éric,24/02/1966,EXG,"(34) - Professeur, profession scientifique",NaN,M,DUPONT,Jean Claude,06/07/1960,NaN
1,1,Ain,101,1ère circonscription,2,4,M,MAÎTRE,Christophe,22/04/1969,RN,(22) - Commerçant et assimilé,NaN,M,MOUREY,Jean-Marie,03/01/1965,NaN
2,1,Ain,101,1ère circonscription,3,3,M,BRETON,Xavier,25/11/1962,LR,(33) - Cadre de la fonction publique,OUI,M,FLOCHON,Jean-Yves,15/08/1958,NaN
3,1,Ain,101,1ère circonscription,4,5,M,GUERAUD,Sébastien,09/09/1973,UG,"(34) - Professeur, profession scientifique",NaN,F,DIMBERTON,Gaëlle,12/01/1984,NaN
4,1,Ain,101,1ère circonscription,5,1,M,VINCENT,Cyril,21/10/1982,DSV,(23) - Chef d'entreprise de 10 salariés ou plus,NaN,M,VINCENT,Christian,30/08/1955,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,ZZ,Français établis hors de France,ZZ11,11ème circonscription,11,7,M,CHEMINADE,Jacques,20/08/1941,DIV,(74) - Ancien cadre,NaN,F,BIERRE,Christine,19/04/1950,NaN
4006,ZZ,Français établis hors de France,ZZ11,11ème circonscription,12,8,M,ASSELINEAU,François,14/09/1957,DSV,(33) - Cadre de la fonction publique,NaN,F,NGUYEN,Fleur,09/06/1971,NaN
4007,ZZ,Français établis hors de France,ZZ11,11ème circonscription,13,1,F,GENETET,Anne,20/04/1963,ENS,(31) - Profession libérale,OUI,M,PROVENDIER,Remi,24/12/1971,NaN
4008,ZZ,Français établis hors de France,ZZ11,11ème circonscription,14,15,M,CRUCIANI,Jordan,24/09/1990,ECO,(33) - Cadre de la fonction publique,NaN,M,DAPRILE,Rafaele,06/07/2005,NaN


In [3]:
circonscriptions = (
    candidats
    ['Code circonscription']
    .str.replace('ZZ', '99')
    .str.replace('ZX', '977')
    .str.pad(5, fillchar='0')
    .drop_duplicates()
    .reindex()

)

circonscriptions

0       00101
7       00102
13      00103
21      00104
28      00105
        ...  
3944    09907
3953    09908
3964    09909
3983    09910
3995    09911
Name: Code circonscription, Length: 577, dtype: object

In [4]:
def get_inscrits_t1(circonscription):
    url = f'https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2022/resultatsT1/{circonscription[:3]}/{circonscription}.xml'
    
    try:
        df = (
            pd
            .read_xml(
                url,
                xpath='.//Inscrits'
            )
            .assign(CodCirc=circonscription)
        )
    except:
        print(f'error: {circonscription}')
        df = None
        
    
    return df

inscrits_2022_t1 = pd.concat([ get_inscrits_t1(circonscription) for circonscription in circonscriptions.to_list() ])

In [5]:
# def get_resultats_t1(circonscription):
#     url = f'https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2022/resultatsT1/{circonscription[:3]}/{circonscription}.xml'
    
#     try:
#         df = (
#             pd
#             .read_xml(
#                 url,
#                 xpath='.//Candidat'
#             )
#             .assign(CodCirc=circonscription)
#         )
#     except:
#         print(f'error: {circonscription}')
#         df = None
        
    
#     return df
    
    
# resultats_2022_t1 = pd.concat([ get_resultats_t1(circonscription) for circonscription in circonscriptions.to_list() ])

resultats_2022_t1 = pd.read_csv('lg2022_t1_resultats.csv')

resultats_2022_t1

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc,CodCirc1,CodCirc2
0,1,GUILLERMIN,Vincent,M.,ENS,Ensemble ! (Majorité présidentielle),8071,"19,30","9,36",non,00101,00101,01001
1,2,LAHY,Éric,M.,DXG,Divers extrême gauche,391,"0,94","0,45",non,00101,00101,01001
2,3,BRETON,Xavier,M.,LR,Les Républicains,10599,"25,35","12,30",Bal.,00101,00101,01001
3,4,PIROUX GIANNOTTI,Brigitte,Mme,RN,Rassemblement National,8971,"21,46","10,41",non,00101,00101,01001
4,5,MENDES,Michael,M.,DSV,Droite souverainiste,641,"1,53","0,74",non,00101,00101,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6285,4,TAPAYEVA,Tamila,Mme,DSV,Droite souverainiste,481,"1,74","0,49",non,09911,09911,ZZ011
6286,5,MARTIN,Catya,Mme,LR,Les Républicains,1760,"6,37","1,78",non,09911,09911,ZZ011
6287,6,VIDAL,Dominique,Mme,NUP,Nouvelle union populaire écologique et sociale,6849,"24,79","6,93",Bal.,09911,09911,ZZ011
6288,7,BURLOTTE,Olivier,M.,RN,Rassemblement National,742,"2,69","0,75",non,09911,09911,ZZ011


In [6]:
(
    resultats_2022_t1
    .rename(columns=
        {
            'CodNua': 'Code nuance',
            'NomPsn': 'Nom du candidat'
        }
    )
    .pivot_table(
        index = 'CodCirc',
        columns = 'Code nuance',
        values = 'Nom du candidat',
        aggfunc= lambda x : len(x) >= 1
    )
    #.to_csv('lg2022_nuances_t1.csv')
)

Code nuance,DIV,DSV,DVC,DVD,DVG,DXD,DXG,ECO,ENS,LR,NUP,RDG,REC,REG,RN,UDI
CodCirc,,,,,,,,,,,,,,,,
00101,NaN,True,NaN,NaN,NaN,NaN,True,True,True,True,True,NaN,True,NaN,True,NaN
00102,True,True,NaN,True,NaN,NaN,True,True,True,True,True,NaN,True,NaN,True,NaN
00103,True,True,True,NaN,NaN,NaN,True,True,True,True,True,NaN,True,NaN,True,NaN
00104,NaN,True,True,True,True,NaN,True,True,True,True,True,True,True,NaN,True,NaN
00105,True,True,NaN,True,True,NaN,True,NaN,NaN,True,True,NaN,True,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98701,True,NaN,NaN,True,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN
98702,NaN,True,NaN,True,NaN,NaN,True,True,True,NaN,NaN,NaN,NaN,True,True,NaN
98703,NaN,NaN,NaN,True,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN


In [7]:
def ecarts(x):
    l = x.sort_values(ascending=False).to_list()
    return l[0] - l[1] if len(l) > 1 else None

ecarts_2022_t1 = (
    resultats_2022_t1
    .assign(NbVoix = lambda df: df.NbVoix.astype(int) )
    .groupby('CodCirc')
    .agg({'NbVoix': ecarts})
)

ecarts_2022_t1.sort_values('NbVoix').head(50)

,NbVoix
CodCirc,
05905,2
04602,13
08103,27
03403,41
07105,44
08002,51
07706,58
08603,60
09406,65


In [8]:
# def get_resultats_t2(circonscription):
#     url = f'https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2022/resultatsT2/{circonscription[:3]}/{circonscription}.xml'
    
#     try:
#         df = (
#             pd
#             .read_xml(
#                 url,
#                 xpath='.//Tour[./NumTour = "2"]//Candidat'
#             )
#             .assign(CodCirc=circonscription)
#         )
#     except:
#         print(f'error: {circonscription}')
#         df = None
        
    
#     return df
    
    
# resultats_2022_t2 = pd.concat([ get_resultats_t2(circonscription) for circonscription in circonscriptions.to_list() ])

resultats_2022_t2 = pd.read_csv('lg2022_t2_resultats.csv')

resultats_2022_t2

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc,CodCirc1,CodCirc2
0,3,BRETON,Xavier,M.,LR,Les Républicains,24408,"63,22","28,32",oui,00101,00101,01001
1,8,GUÉRAUD,Sébastien,M.,NUP,Nouvelle union populaire écologique et sociale,14203,"36,78","16,48",non,00101,00101,01001
2,1,LAPRAY,Lumir,Mme,NUP,Nouvelle union populaire écologique et sociale,17824,"41,66","17,83",non,00102,00102,01002
3,2,DAUBIÉ,Romain,M.,ENS,Ensemble ! (Majorité présidentielle),24960,"58,34","24,97",oui,00102,00102,01002
4,3,GIVERNET,Olga,Mme,ENS,Ensemble ! (Majorité présidentielle),18398,"58,72","22,38",oui,00103,00103,01003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,3,BEN CHEÏKH,Karim,M.,NUP,Nouvelle union populaire écologique et sociale,11348,"54,07","9,41",oui,09909,09909,ZZ009
1144,4,LAKRAFI,Amélia,Mme,ENS,Ensemble ! (Majorité présidentielle),13048,"63,58","12,49",oui,09910,09910,ZZ010
1145,9,MOUSSA,Chantal,Mme,NUP,Nouvelle union populaire écologique et sociale,7474,"36,42","7,16",non,09910,09910,ZZ010
1146,1,GENETET,Anne,Mme,ENS,Ensemble ! (Majorité présidentielle),16537,"61,73","16,73",oui,09911,09911,ZZ011


In [9]:
gagnants_2022_t2 = (
    resultats_2022_t2
    .pivot_table(
        index = 'CodCirc',
        columns = 'CodNua',
        values = 'NbVoix'
    )
    .idxmax(axis=1)
    .to_frame(name="En tête")
)

(
    gagnants_2022_t2
    .value_counts()
)

En tête
ENS        244
NUP        127
RN          89
LR          61
DVG         22
DVD         10
REG         10
DVC          4
UDI          3
DIV          1
DSV          1
Name: count, dtype: int64

In [10]:
ecarts_2022_t2 = (
    resultats_2022_t2
    .assign(NbVoix = lambda df: df.NbVoix.astype(int) )
    .groupby('CodCirc')
    .agg({'NbVoix': ecarts})
)

ecarts_2022_t2.sort_values('NbVoix').query('NbVoix <= 100')

,NbVoix
CodCirc,
03106,4.0
07708,4.0
04505,11.0
98601,16.0
09105,18.0
97501,19.0
01601,24.0
06206,56.0
02103,66.0
